In [ ]:
import numpy as np

def trend_sliding_window(data, window_size, horizon):
    """
    Create input/output pairs for trend prediction using a sliding window approach.
    Args:
        data (array-like): The time series data.
        window_size (int): The size of the sliding window.
        horizon (int): The prediction horizon indicating how many steps ahead to predict.

    Returns:
        tuple: A tuple containing the input sequences (X) and the corresponding output values (y).
    """
    X, y = [], []
    for i in range(len(data) - window_size - horizon ):
        input_window = data[i:i+window_size]
        X.append(input_window)      # Input Sequence
        output_window = data[ i + window_size  : i + window_size  + horizon]
        trend_value   = data[ i + window_size - 1  + horizon] - data[ i + window_size + 1]
        y.append(output_window)     # Output Sequence
    return np.array(X), np.array(y)


def sliding_window_th(data, window_size, horizon, th ):
    X, y = [], []
    for i in range(len(data) - window_size - horizon ):
        input_window = data[i:i+window_size]
        trend_value  = data[ i + window_size + horizon - 1 ] - data[ i + window_size - 1]
        if ( trend_value > th):
          y_trend =  1
        elif ( trend_value < -1*th):
          y_trend = -1
        else:
          y_trend = 0
        X.append(input_window)      # Input Sequence
        y.append(y_trend)     # Output Sequence
    return np.array(X), np.array(y)



####



In [ ]:
W  = 10  # input window
H  = 1   # future window , check H  = 1
th = 0.5
ts_data =  np.arange(100) #
##
X , Y         = trend_sliding_window(ts_data, W, H  ) # normal sliding window
X_th , Y_th   = sliding_window_th   (ts_data, W, H , th ) # threshold sliding window


# trend_window_th =


In [ ]:
print_op = 'th' # th , reg
## regular sliding window
print('ts example:' , ts_data[:30], 'method' , print_op)

if print_op == 'reg':
  X_sample , Y_sample = X, Y
if print_op == 'th':
  X_sample , Y_sample = X_th, Y_th

index = 0
for x,y in zip(X_sample , Y_sample):
  print(f'####### time step: {index} ######')
  print('input:' , x )
  print('output:', y)
  if index > 10:
    break
  index +=1


In [ ]:
## example : create ouptuts for diff threshold values
y_th_arr = []
th_arr = np.arange(0,5,0.1)
for th in th_arr:
  y_th = sliding_window_th(ts_data, W, H , th )[1]
  y_th_arr.append(y_th)


In [ ]:
## full example: grid search over W , H , th
W_max  =  60
W_step =  3 # min W = 1
H_max  =  30
H_step =  2 # min H = 1
th_max =  10
th_step = 0.5
y_th_arr = []
X_arr_W    = []
Y_arr_H    = []
Y_arr_H_step    = []

### Optimize over Validation dataset: Creating grid arrays
for h in range(1,H_max,H_step):
  for w in range(1,W_max,W_step):
    for th in (1, th_max , th_step):
      pass #  complete the code: three loops over W, H , th



In [ ]:
import numpy as np

def weighted_accuracy(y_true, y_pred):
    # Total number of samples
    L = len(y_true)
    # Number of samples where y_i = 0
    S = np.sum(np.array(y_true) == 0)

    # Weight for class 0
    weight_0 = (L - S) / L if S != 0 else 1

    # Initialize variables for correct classifications
    correct_classifications = 0

    # Iterate through true and predicted labels
    for yt, yp in zip(y_true, y_pred):
        # If the labels match
        if yt == yp:
            # If the label is 0, apply the custom weight
            if yt == 0:
                correct_classifications += weight_0
            # If the label is 1 or -1, apply a weight of 1
            else:
                correct_classifications += 1

    # Compute the final weighted accuracy
    accuracy = correct_classifications / L

    return accuracy


In [ ]:
### Example if working with tensorflow:

import tensorflow as tf

def weighted_loss(y_true, y_pred):
    # Determine the total number of samples (L) and the count of zeros (S)
    L = tf.shape(y_true)[0]
    S = tf.reduce_sum(tf.cast(tf.equal(y_true, 0), tf.float32))

    # Weight for class 0
    weight_0 = (L - S) / L if S != 0 else 1

    # Calculate the weights for each element
    weights = tf.where(tf.equal(y_true, 0), weight_0, 1.0)

    # Calculate the standard binary cross-entropy loss
    standard_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)

    # Apply the weights
    weighted_loss = weights * standard_loss

    # Return the mean loss
    return tf.reduce_mean(weighted_loss)

# Define your model as usual
model = tf.keras.Sequential([
    # ... your layers here ...
])

# Compile the model with the custom loss function
# model.compile(optimizer='adam', loss=weighted_loss, metrics=['accuracy'])

# Train the model
# model.fit(x_train, y_train, epochs=5)
